In [1]:
# Gautam Jain, Jannis Horn 

%matplotlib notebook
import time
import numpy as np
import matplotlib as plt
import torch  
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as topt
import wandb

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.manual_seed( 666 )

In [2]:
# hinge loss
def hingeLoss( pred, label ):

# softmax

def softmaxLoss( pred, label ):
    

In [3]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

print( "Input Shape: {}".format( trainset[0][0].shape ) )
print( "Training Set: {}, Test Set: {}".format( len(trainset), len(testset) ) )

#print( trainset[0][0] )

dim_in = 3*32*32
dim_out = 10

Files already downloaded and verified
Files already downloaded and verified
Input Shape: torch.Size([3, 32, 32])
Training Set: 50000, Test Set: 10000


In [4]:
def logTraining( epoch, train_loss, time ):
    wandb.log( {"train_loss": train_loss, "train_time": time}, step=epoch )
    
def logTest( epoch, test_loss, acc, conf_mat, time ):
    wandb.log( {"test_loss": test_loss,
                "accuracy": acc, 
                "conf_mat": [wandb.Image(conf_mat, caption="Confussion Matrix")],
                "test_time": time},
               step=epoch)

In [5]:
class LinRegClassifier( nn.Module ):
    
    def __init__( self, layer_list, dropout, act ):
        super( LinRegClassifier, self ).__init__()
        self.act_f = self.stringToFunc( act )
        self.dropout = dropout
        self.buildNet( layer_list )
        
    def stringToFunc( self, act_str ):
        if act_str in ["relu", "ReLU"]:
            return nn.ReLU
        elif act_str in ["sig", "sigmoid", "Sigmoid"]:
            return nn.Sigmoid
        elif act_str in ["tanh", "Tanh"]:
            return nn.Tanh
        else:
            raise RuntimeError( "Unknown act_str" )
        
    def buildNet( self, layer_list ):
        self.layers = nn.ModuleList()
        for it, dim_in in enumerate(layer_list[:-2]):
            self.layers.append( self.buildLayer( dim_in, layer_list[it+1], self.dropout, self.act_f ) )
        self.layer_out = nn.Linear( layer_list[-2], layer_list[-1] )
            
    def buildLayer( self, dim_in, dim_out, dropout, act_f ):
        return nn.Sequential( nn.Linear( dim_in, dim_out ),
                              nn.Dropout( dropout ),
                              act_f() )
    
    def save( self, f ):
        torch.save(self.state_dict(), "{}.th".format(f))
        
    def load( self, f ):
        self.load_state_dict(torch.load( "{}.th".format(f) ))
        
    
    def forward( self, x ):
        for layer in self.layers:
            x = layer(x)
        return self.layer_out(x)
    
    
    def getGradientNorm( self ):
        out = []
        for p in self.parameters():
            out.append( p.data.norm(2) )
        return out
    
    def getL1Norm( self ):
        n = torch.tensor( 0.0 ).to( device )
        for l in self.layers:
            n += l[0].weight.abs().sum()
        n += self.layer_out.weight.abs().sum()
        return n
    
    def getL2Norm( self ):
        n = torch.tensor( 0.0 ).to( device )
        for l in self.layers:
            n += l[0].weight.square().sum()
        n += self.layer_out.weight.square().sum()
        return n
    
    def wandbConfig( self ):
        wandb.config.act_f = str(self.act_f())
        wandb.config.dropout = self.dropout
        wandb.config.ct_layers = len(self.layers) +1
        for it, layer in enumerate(self.layers):
            wandb.config.update( {"layer_{}".format(it): str(layer[0].out_features)} )
        
    
layers = [dim_in,128,128,dim_out]
dropout = 0.2
act = "relu"

Net = LinRegClassifier( layers, dropout, act )
print(Net)
#print(Net.getGradientNorm())

LinRegClassifier(
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=3072, out_features=128, bias=True)
      (1): Dropout(p=0.2, inplace=False)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Dropout(p=0.2, inplace=False)
      (2): ReLU()
    )
  )
  (layer_out): Linear(in_features=128, out_features=10, bias=True)
)


In [10]:
class Runner:
    def wandbConfig( self ):
        self.model.wandbConfig()
        wandb.config.optimizer = self.optimizer
        wandb.config.regularizer = "lambdas={}".format( self.reg_w )
        wandb.config.dataset = "Cifar10"
    
    
    def initRun( self, model, batch_size, dev, opt, lr, momentum, reg ):
        self.model = model
        self.dev = dev
        self.model.to( dev )
        if opt in [topt.Adagrad, topt.Adadelta, topt.Adam]:
            self.optimizer = opt( model.parameters(), lr=lr )
        else:
            self.optimizer = opt( model.parameters(), lr=lr, momentum=momentum )
        self.loss_func = nn.CrossEntropyLoss().to( dev )
        self.reg_w = torch.Tensor([reg[0], reg[1]]).to( dev )
        self.tr_size = len( trainset )
        self.te_size = len( testset )
        self.loader_training = torch.utils.data.DataLoader( dataset=trainset, 
                                                            batch_size=batch_size, 
                                                            shuffle=True,
                                                            num_workers=2 )
        self.loader_eval = torch.utils.data.DataLoader( dataset=testset, 
                                                        batch_size=batch_size, 
                                                        shuffle=False,
                                                        num_workers=2 )
            
    
    def testModel( self ):
        self.model.eval()
        with torch.no_grad():
            st_pt = time.time()
            conv_mat = np.zeros([10,10])
            corr = 0
            bt_loss = 0.0
            for (x, label) in self.loader_eval:
                x = x.view( -1, dim_in ).to( self.dev )
                label = label.to( self.dev )
                out = self.model(x)
                loss = self.loss_func( out, label )
                bt_loss += loss.cpu().item()
                
                _, pred = torch.max( out.data, 1 )
                ls = label.cpu().numpy()
                pr = pred.cpu().numpy()
                for it in range( label.size(0) ):
                    conv_mat[ls[it], pr[it]] += 1
                corr += (pred.cpu() == label.cpu()).sum().float()
                
            ts_time = time.time() -st_pt
        return bt_loss /len( self.loader_eval ), corr/self.te_size, 10*conv_mat/self.te_size, ts_time

        
    def trainModel( self ):
        self.model.train()
        st_pt = time.time()
        bt_loss = 0.0
        for it, (x, label) in enumerate( self.loader_training ):
            self.optimizer.zero_grad()
            x = x.view(-1,dim_in).to( self.dev )
            label = label.to( self.dev )

            out = self.model(x)
            loss = self.loss_func(out, label)
            norm_1 = self.model.getL1Norm()
            norm_2 = self.model.getL2Norm()
            loss += self.reg_w[0] *norm_1 + self.reg_w[1] *norm_2
            loss.backward()
            bt_loss += loss.cpu().item()

            self.optimizer.step()

        tr_time = time.time() -st_pt
        return bt_loss /len( self.loader_training ), tr_time
    
    
    def __call__( self, model, config, num_epoch, key="" ):
        run = wandb.init( project="cudavision3", name=key, reinit=True )
        with run:
            self.initRun( model, config["bs"], config["dev"], 
                          config["opt"], config["lr"], config["mom"],
                          config["reg"] )
            self.wandbConfig()
            for epoch in range(num_epoch):
                tr_l, tr_time = self.trainModel()
                logTraining( epoch, tr_l, tr_time )
                ts_l, acc, cmat, ts_time = self.testModel()
                logTest( epoch, ts_l, acc, cmat, ts_time )
        del model
        

In [19]:
#SGD, Adam, Adagrad, Adadelta, RMSprop
opts = [topt.Adam, topt.Adagrad, topt.Adadelta, topt.SGD, topt.RMSprop]

dropouts = [0,0.2,0.5]
act = ["relu", "sig", "tan"]
#layers = [dim_in,1024,1024,1024,768,512,512,dim_out]
layers = [dim_in,256,256,256,256,256,dim_out]

cfg = { "bs": 1000, "dev": device, "opt": opts[0], 
        "lr": 5*10e-4, "mom": 0.5, "reg": (10e-6, 10e-5) }

In [ ]:
test_model = LinRegClassifier( layers, dropouts[1], act[0] )
test_model.save( "test_model" )
runner = Runner()
test_epochs = 400
for it, opt in enumerate(opts):
    model = LinRegClassifier( layers, dropouts[1], act[0] )
    model.load( "test_model" )
    cfg["opt"] = opt
    runner( model, cfg, test_epochs, key="optimizer_{}".format(it) )
cfg["opt"] = opts[0]
    
for it, a in enumerate(act[1:]):
    model = LinRegClassifier( layres, dropouts[1], a )
    model.load( "test_model" )
    runner( model, cfg, test_epochs, key="act_f_{}".format(it) )    

for it, dps in enumerate(dropouts):
    model = LinRegClassifier( layres, dps, act[0] )
    model.load( "test_model" )
    runner( model, cfg, test_epochs, key="dropout_{}".format(it) )

wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


train_loss,1.7805
train_time,2.18514
test_loss,1.63721
accuracy,0.4212
test_time,0.51447
_step,399
_runtime,1073
_timestamp,1607085132


train_loss,█▅▂▃▃▂▂▂▂▁▁▁▂▂▁▂▂▂▁▁▂▁▂▂▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁
train_time,▆▆▅▅█▅▆▆▆▆█▆▁▃▃▆▆▁▄▃▅▅█▄▂▅█▄▄▅▄▄▅▅▆▆▆▇▁▅
test_loss,█▄▃▂▃▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▁▃▂▁▂▂▂▂▂▁▁▂▂▂▂▂▂
accuracy,▁▅▇▇▆▇██▇████▇▇▇▇▇▇▇▇███▇▇█▇▇▇▇▇▇██▇█▇▇█
test_time,▅▄▆▄▄▃▅▇▆▆▂▅▂▄▄▂▆▁▂▁▅▃█▂▂▅▄▃▂▄█▄█▆▃▃▆▄▃▆
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [12]:
runner = Runner()
cfg = { "bs": 10000, "dev": device, "opt": opts[2], 
        "lr": 10e-5, "mom": 0.5, "reg": regs[0] }

layers=[dim_in,2048,2048,1024,1024,256,256,64,dim_out]
model = LinRegClassifier( layers, dropouts[1], act[0] )
runner( model, cfg, 1000 )

TypeError: 'float' object is not subscriptable